In [1]:
from parameters import get_parameters
import pandas as pd
from pathlib import Path
from unidecode import unidecode

In [2]:
params = get_parameters()

start_year = params["start_year"]
end_year = params["end_year"]
source_raw = params["source_raw"]
source_curated = params["source_curated"]

In [3]:
PROJECT_ROOT = Path().resolve().parent
print(f"Project root: {PROJECT_ROOT}")

Project root: C:\Users\dxnin\OneDrive\Documentos\dengue-forecast


In [4]:
df_list = []

for year in range(start_year, end_year + 1):
    path_dengue = f"{PROJECT_ROOT}/{source_raw}/Dengue_{year}.xlsx"
    df_dengue = pd.read_excel(path_dengue, engine="openpyxl")
    df_list.append(df_dengue)

for year in range(start_year, end_year + 1):
    path_severe = f"{PROJECT_ROOT}/{source_raw}/Dengue_grave_{year}.xlsx"
    df_severe = pd.read_excel(path_severe, engine="openpyxl")
    df_list.append(df_severe)

df = pd.concat(df_list, ignore_index=True)
print(df.head())

       Separación  CONSECUTIVE2  COD_EVE     FEC_NOT  SEMANA   ANO  \
0  Datos_2007_210     1665806.0      210  2007-03-20      12  2007   
1  Datos_2007_210     1665808.0      210  2007-03-17      12  2007   
2  Datos_2007_210     1665809.0      210  2007-03-12      12  2007   
3  Datos_2007_210     1665810.0      210  2007-03-23      12  2007   
4  Datos_2007_210     1665811.0      210  2007-03-20      13  2007   

      COD_PRE  COD_SUB  EDAD  UNI_MED  ... Pais_residencia  \
0  7300100871        1    35        1  ...             NaN   
1  7300100871        1    14        1  ...             NaN   
2  7300100871        1    14        1  ...             NaN   
3  7300100948        2     6        1  ...             NaN   
4  5400100603        1    12        1  ...             NaN   

  Departamento_residencia Municipio_residencia  Departamento_Notificacion  \
0                  TOLIMA               COELLO                     TOLIMA   
1                  TOLIMA               IBAGUE      

In [5]:
# Filter by confirmed cases
df_cases = df[df["confirmados"] == 1]

# Filter by infections in Colombia
df_cases = df_cases[df_cases["Pais_ocurrencia"] == "COLOMBIA"]

# Filter by department and municipios (first clean the strings)
df_cases["Departamento_ocurrencia"] = df_cases["Departamento_ocurrencia"].apply(lambda x: unidecode(str(x))).str.strip().str.upper()
df_cases["Municipio_ocurrencia"] = df_cases["Municipio_ocurrencia"].apply(lambda x: unidecode(str(x))).str.strip().str.upper()

df_cases = df_cases[df_cases["Departamento_ocurrencia"] == "ANTIOQUIA"]
df_cases = df_cases[df_cases["Municipio_ocurrencia"].isin(["MEDELLIN"])]

print(df_cases.head())

           Separación  CONSECUTIVE2  COD_EVE     FEC_NOT  SEMANA   ANO  \
10768  Datos_2007_210     1665727.0      210  2007-04-06      18  2007   
18269  Datos_2007_210     1417878.0      210  2007-07-10      28  2007   
35416  Datos_2008_210     2166671.0      210  2008-11-13      15  2008   
35537  Datos_2008_210     2170738.0      210  2008-01-05      35  2008   
36150  Datos_2008_210     2175078.0      210  2008-09-25       8  2008   

          COD_PRE  COD_SUB  EDAD  UNI_MED  ... Pais_residencia  \
10768  5000100482        0    49        1  ...             NaN   
18269  4700100597        6    50        1  ...             NaN   
35416   500108199        1    60        1  ...             NaN   
35537   500102178        6    52        1  ...             NaN   
36150   500105890        1    46        1  ...             NaN   

      Departamento_residencia Municipio_residencia  Departamento_Notificacion  \
10768               ANTIOQUIA             MEDELLIN                       META

In [6]:
# FEC_CON to datetime type
df_cases["FEC_CON"] = pd.to_datetime(df_cases["FEC_CON"]).dt.date
df_cases["FEC_CON"] = pd.to_datetime(df_cases["FEC_CON"], format="%Y-%m-%d")

# Group dataset by infected date and place
df_cases = df_cases.groupby(["FEC_CON", "Municipio_ocurrencia"]).agg({"COD_EVE" : "size"})

# Index as columns
df_cases = df_cases.reset_index(drop=False)

# Change column names
df_cases = df_cases.rename(columns={"FEC_CON":"DATE", "COD_EVE": "CASES", "Municipio_ocurrencia":"MUNICIPIO"})

In [7]:
# Filter by date range
df_cases = df_cases[(df_cases["DATE"] >= f"{start_year}-01-01") & (df_cases["DATE"] <= f"{end_year}-12-31")]

# Change date to the start of the week
date_range = pd.date_range(start=F"{start_year}-01-01", end=f"{end_year}-12-31", freq="W")
df_cases["DATE"] = df_cases["DATE"].apply(lambda d: date_range[date_range <= d].max())
df_cases = df_cases.dropna(subset=["DATE"])

# Group records by week
df_cases = df_cases.groupby(["DATE", "MUNICIPIO"]).agg({"CASES": "sum"})
df_cases = df_cases.reset_index(drop=False)

# Add missing weeks
municipios = df_cases["MUNICIPIO"].unique()
weeks_x_municipios = pd.MultiIndex.from_product([date_range, municipios], names=["DATE", "MUNICIPIO"]).to_frame(index=False)
df_cases = df_cases.merge(weeks_x_municipios, on=["DATE", "MUNICIPIO"], how="right")
df_cases["CASES"] = df_cases["CASES"].fillna(0)

# Change column CASES to int
df_cases["CASES"] = df_cases["CASES"].astype(int)

# Format column MUNICIPIO
df_cases["MUNICIPIO"] = df_cases["MUNICIPIO"].replace("MEDELLIN", "MEDELLÍN")

In [8]:
# Show dataframe info
print("Total of records:", len(df_cases))
print("============================================")
print("Columns and data types:")
print(df_cases.dtypes)

Total of records: 887
Columns and data types:
DATE         datetime64[ns]
MUNICIPIO            object
CASES                 int64
dtype: object


In [9]:
df_cases.head()

,DATE,MUNICIPIO,CASES
0,2007-01-07,MEDELLÍN,0
1,2007-01-14,MEDELLÍN,0
2,2007-01-21,MEDELLÍN,0
3,2007-01-28,MEDELLÍN,0
4,2007-02-04,MEDELLÍN,0


In [10]:
pd.to_pickle(df_cases, f"{PROJECT_ROOT}/{source_curated}/dengue.pkl")